In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

# temperature

Defaults to 1

사용할 샘플링 온도 값이 높을수록 모델이 더 많은 위험을 감수 할 것입니다. 보다 창의적인 응용 프로그램은 0.9를 시도하고 잘 정의 된 답변이있는 경우 0 (Argmax 샘플링)을 시도하십시오.

일반적으로 Temperature 또는 Top_p 값을 바꾸는걸 추천하지만, 둘 다 바꾸는건 아닙니다.

In [4]:
def call_openai(num_times, messages, temperature):
    for i in range(num_times):
        
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=60,
            temperature=temperature
        )
        
        print(str(i+1) + ": " + response.choices[0].message.content.strip(" \n"))

In [5]:
messages=[
    {"role": "system", "content": "You are a helpful  assistant."},
    {"role": "user", "content": 'The best pet is a '}
]

call_openai(5, messages, temperature = 0)

1: subjective question and depends on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, playful, and can provide companionship. Others may prefer cats because they are independent, low-maintenance, and can be affectionate. Some people may prefer smaller pets like hamsters or
2: subjective question and depends on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, playful, and can provide companionship. Others may prefer cats because they are independent, low-maintenance, and can be affectionate. Some people may prefer smaller pets like hamsters or
3: subjective question and depends on individual preferences and circumstances. Some people may prefer dogs as pets because they are loyal, playful, and can provide companionship. Others may prefer cats because they are independent, low-maintenance, and can be affectionate. Some people may prefer smaller pets like hamsters or
4: subjective qu

In [6]:
call_openai(5, messages, temperature = 1)

1: subjective topic and will ultimately depend on an individual's personal preferences and lifestyle. Some popular choices for pets include dogs, cats, fish, birds, reptiles, and small mammals like rabbits or guinea pigs. It is important to consider factors such as the amount of time and attention you can dedicate
2: subjective topic, as it depends on individual preferences and circumstances. However, some popular pets include dogs, cats, fish, birds, and rabbits. Consider factors such as your lifestyle, living conditions, time commitment, and allergies when choosing a pet.
3: subjective matter and ultimately depends on personal preferences and lifestyles. Some people may prefer the companionship of a dog, while others may prefer the independence of a cat. Other popular pets include birds, fish, rabbits, guinea pigs, and hamsters. It is important to consider factors such as the
4: subjective statement, as different people have different preferences and lifestyles. Some may argue that d

# top_p

Defaults to 1

An alternative to sampling with temperature, called nucleus sampling, where the model considers the results of the tokens with top_p probability mass. 따라서 0.1은 상위 10% 확률 질량을 포함하는 토큰 만 고려된다는 것을 의미합니다.

일반적으로 Temperature 또는 Top_p 값을 바꾸는걸 추천하지만, 둘다 바꾸는건 아닙니다.

In [7]:
def call_openai(num_times, messages, top_p):
    for i in range(num_times):
        
        response = client.chat.completions.create(
            model=deployment_name,
            messages=messages,
            max_tokens=60,
            top_p = top_p
        )

        print(str(i+1) + ": " + response.choices[0].message.content.strip(" \n"))

In [8]:
messages=[
    {"role": "user", "content": 'The best pet is a '},
]

call_openai(10, messages, top_p = 1)

1: subjective question, as it depends on personal preferences and lifestyles. Some people may consider dogs to be the best pets because they are loyal, loving, and provide companionship. Others may prefer cats because they are independent, low maintenance, and known to be great for small living spaces. Some may argue
2: subjective question and can vary depending on individual preferences and lifestyles. Some people may prefer dogs as pets, as they are known for their loyalty and companionship. Others may prefer cats for their independence and low maintenance. Small animals like hamsters or rabbits can also be great pets for those who have limited
3: subjective question, as the best pet will vary depending on individual preferences, lifestyle, and living situation. Some people may consider dogs to be the best pet due to their loyalty, companionship, and ability to be trained. Others may prefer cats for their independent nature and low maintenance requirements. Some may
4: subjective opi

In [21]:
call_openai(10, messages, top_p = 0.1)

1: subjective matter and depends on individual preferences and lifestyles. Some people may prefer dogs as they are loyal and make great companions, while others may prefer cats as they are independent and low-maintenance. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one
2: subjective matter and depends on individual preferences and lifestyles. Some people may prefer dogs as they are loyal and make great companions, while others may prefer cats as they are independent and low-maintenance. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one
3: subjective matter and depends on individual preferences and lifestyles. Some people may prefer dogs as they are loyal and make great companions, while others may prefer cats as they are independent and low-maintenance. Other popular pets include birds, fish, rabbits, and hamsters. Ultimately, the best pet is one
4: subjective matter and depends on individual pr

# n

Defaults to 1

요청한 Prompt에 대해 Completion 몇 개를 생성할지 정하는 옵션임.

참고 :이 매개 변수는 많은 Completion을 생성하기 때문에 토큰 할당량을 빠르게 소비 할 수 있습니다. 신중하게 사용하고 max_tokens에 대한 합리적인 설정이 있는지 확인하고 중지하십시오.

In [15]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    max_tokens=60,
    n=2
)

i=0
for resp in response.choices:
    print(resp.message.content.strip(" \n"))

matter of personal preference. Some people may argue that the best pet is a dog because they are loyal, loving, and can be trained to perform various tasks. Others may argue that cats make the best pets because they are independent, low-maintenance, and provide companionship. Additionally, some people may
subjective opinion and can vary depending on personal preference. Some people may consider dogs to be the best pet because of their loyalty, companionship, and ability to be trained. Others might prefer cats for their independent nature and low-maintenance care. Other popular pets include birds, fish, rabbits, ham


# logprobs

Defaults to null

log probe에 로그 확률을 포함시킬 가능성이 높은 토큰뿐만 아니라 선택한 토큰을 포함합니다. 예를 들어 LogProbs가 5 인 경우 API는 가장 가능성이 높은 5 개의 토큰 목록을 반환합니다. API는 항상 샘플링 된 토큰의 로그 프로브를 반환하므로 응답에 LogProbs+1 요소가 있을 수 있습니다. 신뢰도 평가를 위해 유용하게 사용 가능합니다.

LogProbs의 최대 값은 5입니다.

- 새로운 GPT-35-Turbo 및 GPT-4 모델에서는 logprobs, best_of 및 echo 매개변수를 사용할 수 없습니다. 이러한 매개변수를 설정하면 오류가 발생합니다.
- https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/chatgpt?pivots=programming-language-chat-completions

In [27]:
# response = openai.ChatCompletion.create(
#     engine=model,
#     messages=messages,
#     max_tokens=60,
#     logprobs = 2,
# )

# print(response['choices'][0]['logprobs'])
# print(response['choices'][0]['text'])

# presence_penalty

Defaults to 0

-2.0에서 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트에 나타나는지 여부에 따라 새 토큰에 페널티를 주어 모델이 새 주제에 대해 이야기할 가능성을 높입니다.

# frequency_penalty

Defaults to 0

-2.0에서 2.0 사이의 숫자입니다. 양수 값은 지금까지 텍스트의 기존 빈도를 기반으로 새 토큰에 페널티를 주어 모델이 동일한 줄을 그대로 반복할 가능성을 줄입니다.

# function_call

array

모델이 함수 호출에 응답하는 방식을 제어합니다. '없음'은 모델이 함수를 호출하지 않고 최종 사용자에게 응답함을 의미합니다. '자동'은 모델이 최종 사용자 또는 함수 호출 중에서 선택할 수 있음을 의미합니다. {'name': 'my_function'}을 통해 특정 함수를 지정하면 모델이 해당 함수를 강제로 호출합니다. 함수가 없는 경우 'none'이 기본값입니다. 기능이 있는 경우 '자동'이 기본값입니다.

# logit_bias

Defaults to null

완료에 지정된 토큰이 나타날 가능성을 수정합니다.

토큰(토크나이저의 토큰 ID로 지정됨)을 -100에서 100까지의 관련 바이어스 값에 매핑하는 json 개체를 허용합니다. 수학적으로 바이어스는 샘플링 전에 모델에서 생성된 로짓에 추가됩니다. 정확한 효과는 모델마다 다르지만 -1과 1 사이의 값은 선택 가능성을 낮추거나 높여야 합니다. -100 또는 100과 같은 값은 관련 토큰을 금지하거나 배타적으로 선택해야 합니다.

As an example, you can pass {"50256": -100} to prevent the <|endoftext|> token from being generated.

Reference: OpenAI API Reference [https://platform.openai.com/docs/api-reference/chat/create]